Соединение и аутентификация

При удачном подключении к почте увидим сообщение [b'Authentication successful']

In [1]:
import imaplib
import email
from email.header import decode_header
import base64
from bs4 import BeautifulSoup
import re

def read(sender_of_interest=None):
    # Очищаем файл перед записью
    with open('filtered_emails.txt', 'r+') as f:
        f.truncate(0) 

    # Подключаемся к серверу
    imap = imaplib.IMAP4_SSL("imap.gmail.com", 993)
    imap.login('remboplas@gmail.com', 'trok ekct masw xlwz')
    imap.select('INBOX')

    # Поиск всех писем
    if sender_of_interest:
        status, response = imap.uid('search', None, 'ALL', 'FROM {0}'.format(sender_of_interest))
    else:
        status, response = imap.uid('search', None, 'ALL')

    if status == 'OK':
        unread_msg_nums = response[0].split()
        print(f"Found {len(unread_msg_nums)} emails")  # Отладочная информация
    else:
        unread_msg_nums = []
        print("No emails found")

    # Инвертируем список ID, чтобы получить последние письма первыми
    unread_msg_nums = unread_msg_nums[::-1]

    data_list = []
    count = 0  # Счётчик писем

    for e_id in unread_msg_nums:
        if count >= 400:  # Ограничение на 400 последних писем
            break

        e_id = e_id.decode('utf-8')
        _, response = imap.uid('fetch', e_id, '(RFC822)')

        # Используем message_from_bytes для правильного парсинга писем
        raw_email = response[0][1]
        email_message = email.message_from_bytes(raw_email)

        # Проверка, multipart или нет, чтобы обработать текстовые и HTML части
        if email_message.is_multipart():
            for part in email_message.walk():
                content_type = part.get_content_type()
                if content_type == 'text/plain':  # Извлекаем только текстовую часть
                    body = part.get_payload(decode=True).decode('utf-8', errors='ignore')
                    data_list.append(body)
                    print(f"Added text/plain part from email {e_id}")  # Отладочная информация
                    count += 1
                    break  # Если нашли нужную часть, выходим
        else:
            # Если не multipart, просто получаем тело письма
            body = email_message.get_payload(decode=True).decode('utf-8', errors='ignore')
            data_list.append(body)
            
            count += 1

    # Запись в файл
    with open('filtered_emails.txt', 'w', encoding='utf-8') as f:
        for elem in data_list:
            # Удаляем блоки стилей <style>...</style>
            elem = re.sub(r"<style.*?>.*?</style>", "", elem, flags=re.S)
            elem = re.sub(r"&nbsp;", "", elem, flags=re.S)
            # Удаляем все HTML-теги
            elem = re.sub(r"<[^>]+>", "", elem, flags=re.S)
            
            if 'Высылаю обратную связь' in elem:
                print(f"Added non-multipart email {e_id}")  # Отладочная информация

                # Разбиваем на строки и записываем строки, если они не пустые
                for line in elem.splitlines():
                    if line.strip():  # Проверяем, не пустая ли строка
                        f.write(line + '\n')  # Записываем в файл

                # Добавляем разделитель для каждого письма
                f.write('\n++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++\n')

    # Завершаем работу с сервером
    imap.logout()

# Вызов функции
read()


Found 1695 emails
Added text/plain part from email 4985
Added text/plain part from email 4984
Added text/plain part from email 4983
Added text/plain part from email 4982
Added text/plain part from email 4981
Added text/plain part from email 4980
Added text/plain part from email 4979
Added text/plain part from email 4978
Added text/plain part from email 4977
Added text/plain part from email 4976
Added text/plain part from email 4975
Added text/plain part from email 4974
Added text/plain part from email 4973
Added text/plain part from email 4972
Added text/plain part from email 4971
Added text/plain part from email 4970
Added text/plain part from email 4969
Added text/plain part from email 4968
Added text/plain part from email 4967
Added text/plain part from email 4963
Added text/plain part from email 4962
Added text/plain part from email 4961
Added text/plain part from email 4960
Added text/plain part from email 4959
Added text/plain part from email 4958
Added text/plain part from email

In [2]:
with open('filtered_emails.txt', 'r') as file:
    lines = file.readlines()

text = []  # Список для хранения блоков текста
cnt = 0  # Начальная позиция для нарезки

# Проходим по всем строкам файла
for i in range(len(lines)):
    # Проверяем, если строка содержит только плюсики (ваш разделитель)
    if lines[i].strip() == '++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++':
        # Добавляем блок текста от cnt до i в список text
        text.append(''.join(lines[cnt:i]).strip())
        cnt = i + 1  # Обновляем начало следующего блока

# Добавляем последний блок текста, если он есть после последней строки с плюсами
if cnt < len(lines):
    text.append(''.join(lines[cnt:]).strip())

# print(text[0])
print(text[len(text)-1])


-------- Перенаправленное сообщение --------
Тема: 	ОС по итогам просмотра Мышанского Владимира Викторовича от 
26.09.2024.
Дата: 	Tue, 01 Oct 2024 17:57:40 +0300
От: 	Тернов Сергей 
Отвечать: 	Тернов Сергей 
Кому: 	Алексей Аникеев 
Копия: 	Сергей Викторович Лавриченко , Ваге Акобян 
Добрый день.
Высылаю обратную связь по продавцу Скляров Николай Евгеньевич , название 
ТТ г.Гуково, ул. Комсомольская, д.49 Б, дата: 27.09.2024
В указанный период была: Продажа
*1. Оценка*
Максимальная оценка проверок по критериям – 100 баллов. Итоги просмотра:
Уровень сервиса - 23 из 25
НЭП - 60 из 67
Продажа аксессуаров - 0 из 6
Отработка по акции - 2 из 2
Итоговая сумма баллов с учетом обязательных и не обязательных пунктов - 
85 балов или 85%
*2.Выявленные отклонения, рекомендация*
Продавец не в уставной форме обуви.
Продавец не проверил состояние ткущего АКБ.
Клиенту не было предложено доп. товары. Так же не было демонстрации 
аксессуаров.
Клинту не выдали визитных карточек и листовок.
С уважением, Те

In [6]:
import re
from natasha import MorphVocab, NamesExtractor

# Инициализация морфологического словаря
morph_vocab = MorphVocab()
extractor = NamesExtractor(morph_vocab)

def to_nominative_natasha(full_name):
    """
    Преобразует ФИО из родительного падежа в именительный с помощью Natasha.
    """
    # Получаем первый результат из генератора
    match = next(extractor(full_name), None)
    
    if match:
        # Получаем части имени в именительном падеже
        name = match.fact.as_dict()
        nominative_name = f"{name['last']} {name['first']} {name['middle']}"
        return nominative_name
    return full_name  # Если не удалось преобразовать, возвращаем оригинал

def replace_za_with_fraction(text):
    return re.sub(r'\bза\b', '/', text)

def clean_text(text):
    start_index = text.find('1. Оценка.')
    if start_index != -1:
        text = text[start_index:]
    
    end_index = text.find('2.2.Прошлая оценка')
    if end_index == -1:
        end_index = text.find('С уважением,')
    if end_index != -1:
        text = text[:end_index]

    text = re.sub(r'2\.\s+\.\.\..*?\n', '', text)
    text = re.sub(r'[-]+ Перенаправленное сообщение [-]+\s*Tема:.*?\n', '', text, flags=re.S)

    cleaned_words = []
    for word in text.split():
        if word.lower() not in stop_words or word.isdigit():
            cleaned_words.append(word)

    cleaned_text = ' '.join(cleaned_words)
    cleaned_text = replace_za_with_fraction(cleaned_text)

    return cleaned_text

def extract_seller_info(text):
    match_seller = re.search(r'(по продавцу|Продавец)\s+([^\n,]+)', text)
    match_address = re.search(r'(г\.\s*[^\n,]+,\s*ул\.\s*[^\n,]+,\s*д\.\s*\d+)', text)
    match_date = re.search(r'дата:\s*([^\n,]+)', text)

    if match_seller:
        seller_info = to_nominative_natasha(match_seller.group(2).strip())  # Преобразуем ФИО в именительный падеж
        if match_address:
            seller_info += f", {match_address.group(1).strip()}"
        
        if match_date:
            seller_info += f", дата: {match_date.group(1).strip()}"
        
        return seller_info
    
    return None

def extract_correct_date(text):
    dates = re.findall(r'дата:\s*([^\n,]+)', text)
    
    if dates:
        return dates[-1]
    
    return None

# Новый список для хранения кортежей
cortages = []

for t in text:
    seller = extract_seller_info(t)
    correct_date = extract_correct_date(t)
    
    if seller:
        cleaned_text = clean_text(t)
        
        if 'дата:' not in seller and correct_date:
            seller += f", дата: {correct_date}"
        
        cortage = (seller, cleaned_text)
        cortages.append(cortage)

for c in cortages:
    print(f"Продавец и адрес: {c[0]}, Текст: {c[1]}")


AttributeError: 'Name' object has no attribute 'as_dict'